In [0]:
from pyspark.sql.functions import current_timestamp

def add_ingestion(input_df):
    output_df = input_df.withColumn("ingestion_time", current_timestamp())
    return output_df

In [0]:
def create_or_upsert_managed_delta_table(df, delta_table_catalog_path):
    # Check if the Delta table exists
    tableExists = spark._jsparkSession.catalog().tableExists(delta_table_catalog_path)

    if tableExists:
        # Register the new data as a temporary view
        df.createOrReplaceTempView("updates")
        
        # Perform the merge operation
        spark.sql(f"""
                  MERGE INTO {delta_table_catalog_path} AS existing
                  USING updates
                  ON existing.customer_id = updates.customer_id
                  WHEN MATCHED THEN
                    UPDATE SET *
                  WHEN NOT MATCHED THEN
                    INSERT *
                  """)
        return f"New data has been succesfully merged to table: {delta_table_catalog_path}"
    
    else:
        # If the table does not exist, write the DataFrame as a new Delta table
        df.write.format("delta").mode("overwrite").saveAsTable(delta_table_catalog_path)
        return f"A new table has been succesfully created at: {delta_table_catalog_path}"